In [1]:
pip install qutip

Note: you may need to restart the kernel to use updated packages.


In [2]:
from qutip import *
import numpy as np
import matplotlib.pyplot as plt
from qutip.qip.operations import *
np.set_printoptions(precision = 3)

In [3]:
# spin operators
X = sigmax()
Y = sigmay()
Z = sigmaz()
I = qeye(2)

II = tensor(I, I)

IX = tensor(X, I)
IY = tensor(Y, I)
IZ = tensor(Z, I)

SX = tensor(I, X)
SY = tensor(I, Y)
SZ = tensor(I, Z)

IZSZ = tensor(Z, Z) # J coupling

In [4]:
R = lambda state, angle: (state * -1j * angle).expm()

Ix = lambda angle: R(IX / 2, angle)
Sx = lambda angle: R(SX / 2, angle)
IxSx = lambda angle: R((IX + SX) / 2, angle)

Iy = lambda angle: R(IY / 2, angle)
Sy = lambda angle: R(SY / 2, angle)
IySy = lambda angle: R((IY + SY) / 2, angle)

# in theory we could do Iz and Sz
Iz = lambda angle: R(IZ / 2, angle)
Sz = lambda angle: R(SZ / 2, angle)

# but we cannot pulse on Z in NMR in the same way
# so we do a composite of X and Y pulse
Iz_composite = lambda angle: Ix(np.pi/2) * Iy(angle) * Ix(-np.pi/2)
Sz_composite = lambda angle: Sx(np.pi/2) * Sy(angle) * Sx(-np.pi/2)
IzSz = lambda angle: R(IZSZ / 4, angle)

# we also need
SyIx = lambda angle: R((SY +IX) / 2, angle)
SxIy = lambda angle: R((SX +IY) / 2, angle)

**Pulse Sequence Phases (3rd Param in NMR `pulse(channel, amp, phase, length)`)**\
+x = 0\
+y = 1\
-x = 2\
-y = 3

In [5]:
# helpful global phases to remember
global_imag_phase = np.exp(-1j * np.pi / 2)
global_neg_phase = np.exp(-1j * np.pi)

### Question 1

#### Bit Flip

Pulse Sequence for 1H: 
`pulse(1, a90H, 0, d180H)`\
Pulse Sequence for 13C:
`pulse(2, a90C, 0, d180C)`\
Pulse Sequence for both:
`pulse(1, a90HC, 0, freq1H, 2, a90C, 0, freq13C, d180C)`

In [6]:
def bitflip_operator(qubit):
    if qubit == "1H":
        return Ix(-np.pi) * global_imag_phase
    elif qubit == "13C":
        return Sx(-np.pi)  * global_imag_phase
    elif qubit == "both":
        return IxSx(-np.pi) * global_neg_phase
    else: 
        print(f"Invalid qubit for bitflip. Do nothing.")
        return II

In [7]:
print(bitflip_operator('1H')) 
print(bitflip_operator('13C'))
print(bitflip_operator('both') )

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]
Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]
Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]


#### Hadamard

Pulse Sequence for 1H:

`pulse(1, a90H, 0, d180H)`\
`delay(0.25)`\
`pulse(1, a90H, 1, d90H)`

Pulse Sequence for 13C:

`pulse(2, a90C, 0, d180C)`\
`delay(0.25)`\
`pulse(2, a90C, 1, d180C)`


Pulse Sequence for both:

`pulse(1, a90HC, 0, freq1H, 2, a90C, 0, freq13, d180C)`\
`delay(0.25)`\
`pulse(1, a90HC, 1, freq1H, 2, a90C, 1, freq13, d90C)`

In [8]:
def hadamard_operator(qubit):
    if qubit == "1H":
        return Ix(np.pi) * Iy(np.pi/2) * global_imag_phase * global_neg_phase
    
    elif qubit == "13C":
        return Sx(np.pi) * Sy(np.pi/2) * global_imag_phase * global_neg_phase

    elif qubit == "both":
        return IxSx(np.pi) * IySy(np.pi/2) * global_neg_phase
    
    else: 
        print(f"Invalid qubit for Hadamard. Do nothing.")
        return II

In [9]:
print(np.sqrt(2) * hadamard_operator('1H'))
print(np.sqrt(2) * hadamard_operator('13C'))
print(2 * hadamard_operator('both'))

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.  0.  1.  0.]
 [ 0.  1.  0.  1.]
 [ 1.  0. -1.  0.]
 [ 0.  1.  0. -1.]]
Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.  1.  0.  0.]
 [ 1. -1.  0.  0.]
 [ 0.  0.  1.  1.]
 [ 0.  0.  1. -1.]]
Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.  1.  1.  1.]
 [ 1. -1.  1. -1.]
 [ 1.  1. -1. -1.]
 [ 1. -1. -1.  1.]]


#### CZ

Composite Iz pulse on NMR:

`pulse(1, a90H, 0, d90H)`\
`delay(0.25)`\
`pulse(1, a90H, 1, d90H)`\
`delay(0.25)`\
`pulse(1, d90H, 2, d90H)`\
`delay(0.25)`

Composite Sz pulse on NMR:\
`pulse(2, a90C, 0, d90C)`\
`delay(0.25)`\
`pulse(2, a90C, 1, d90C)`\
`delay(0.25)`\
`pulse(2, a90C, 2, d90C)`

In [10]:
def CZ_operator(NMR=True):
    global_phase = np.exp(1j * np.pi / 4)
    if NMR:
        return Iz_composite(np.pi/2) * Sz_composite(np.pi/2) * IzSz(-np.pi) * global_phase
    else:
        return Iz(np.pi/2) * Sz(np.pi/2) * IzSz(-np.pi)

In [11]:
print(CZ_operator())

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  0. -1.]]


### CNOT

Pulse Sequence for Approximate CNOT:

`pulse(2, a90C, 0, d90C)` \
`delay(dEvolution)` \
`pulse(2, a90C, 1, d90C)`

Pulse Sequence for Actual CNOT:

`pulse(2, a90C, 1, d45C)`\
`delay(0.25)`\
`pulse(2, a90C, 0, d180C)`\
`delay(0.25)`\
`pulse(2, a90C, 3, d45C)`\
`delay(0.25)`

`pulse(1, a90H, 2, d90H)`\
`delay(0.25)`\
`pulse(1, a90H, 1, d90H)`\
`delay(0.25)`\
`pulse(1, a90H, 0, d90H)` \
`delay(0.25)`

`pulse(2, a90C, 2, d90C)`\
`delay(0.25)`\
`pulse(2, a90C, 1, d90C)`\
`delay(0.25)`\
`pulse(2, a90C, 0, d90C)`

`delay(dEvolution)`

`pulse(2, a90C, 1, d45C)`\
`delay(0.25)`\
`pulse(2, a90C, 0, d180C)`\
`delay(0.25)`\
`pulse(2, a90C, 3, d45C)`



In [12]:
def CNOT_operator(approx=False):
    # global_phase = np.exp(1j * np.pi / 4)
    if not approx: 
        return hadamard_operator('13C') * CZ_operator() * hadamard_operator('13C')
    else:
        return Sz(-np.pi/2) * Sx(np.pi/2) * IzSz(np.pi) * Sy(np.pi/2) * Iz(np.pi/2) 

In [13]:
print(CNOT_operator(approx=False))
print(CNOT_operator(approx=True))

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]
Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.707-0.707j 0.   +0.j    0.   +0.j    0.   +0.j   ]
 [0.   +0.j    0.707-0.707j 0.   +0.j    0.   +0.j   ]
 [0.   +0.j    0.   +0.j    0.   +0.j    0.707-0.707j]
 [0.   +0.j    0.   +0.j    0.707-0.707j 0.   +0.j   ]]


Lets plot the expected spectra for each of these gates on our NMR system

In [14]:
# rho thermal (not including 1/4 I)
rho = Qobj([[5,0,0,0],
            [0,3,0,0],
            [0,0,-3,0],
            [0,0,0,-5]], dims = [[2,2], [2,2]])

In [15]:
def evolve_density_matrix(rho, gate):
  evolved_rho = gate * rho * gate.dag()

  a = np.array(evolved_rho)[0,0]
  b = np.array(evolved_rho)[1,1]
  c = np.array(evolved_rho)[2,2]
  d = np.array(evolved_rho)[3,3]

  return (a,b,c,d)

def sum_P0_P1_P2(P0_spectra, P1_spectra, P2_spectra):
    a = P0_spectra[0] + P1_spectra[0] + P2_spectra[0]
    b = P0_spectra[1] + P1_spectra[1] + P2_spectra[1]
    c = P0_spectra[2] + P1_spectra[2] + P2_spectra[2]
    d = P0_spectra[3] + P1_spectra[3] + P2_spectra[3]

    return a,b,c,d


def plot_spectra(spectra_abcd, label, show=True):
  a,b,c,d = spectra_abcd
  x = ['1H peak 2', '1H peak 1', '13C peak 2', '13C peak 1']
  y = [b-d, a-c, c-d, a-b]

  if show:
    plt.bar(x, y)
    plt.xlabel('Frequency')
    plt.ylabel('Signal Amplitude')
    plt.grid(True)
    plt.title(label)
    plt.show()

  return y

def solve_linear_abcd(peaks, state):
    peak1, peak2, peak3, peak4 = peaks
    A = np.array([
        [0, 1, 0, -1],  # for equation b-d = v1
        [1, 0, -1, 0],  # for equation a-c = v2
        [0, 0, 1, -1],  # for equation c-d = v3
        [1, -1, 0, 0]   # for equation a-b = v4
    ])

    B = np.array([peak1, peak2, peak3, peak4])

    # solve linear equations using least squares
    x, residuals, rank, s = np.linalg.lstsq(A, B, rcond=None)

    print(f"Solving for a,b,c,d for|{state}> is approximately:", x.real)


### Question 2

See attached drawings

f(x) = 0: Always outputs 0, regardless of the input.\
f(x) = 1: Always outputs 1, regardless of the input.\
f(x) = x: Outputs whatever the input is.\
f(x) = NOT x: If the input is 0, the output is 1, and vice versa.

Let's implement the quantum circuits:

In [16]:
def U(f):
  if f == 0:
    return II
  if f == 1:
    return bitflip_operator('13C')
  if f == 2:
    return CNOT_operator()
  if f == 3:
    return CNOT_operator() * bitflip_operator('13C')
  

# U0 = (hadamard_operator('13C') * U(0) * hadamard_operator('13C'))
# U1 = (hadamard_operator('13C') * U(1) * hadamard_operator('13C'))
# U2 = (hadamard_operator('13C') * U(2) * hadamard_operator('13C'))
# U3 = (hadamard_operator('13C') * U(3) * hadamard_operator('13C'))


# print(U(0) * global_neg_phase)
# print(U(1) * global_imag_phase * global_neg_phase)
# print(U(2) * global_neg_phase)
# print(U(3) * global_imag_phase * global_neg_phase)

### Question 3: Deutsch Jozsa
We have all of the oracles for DJ algorithm defined above. We also need to do temporal averaging.

In [17]:
def deutsch_josza(oracle):

    if oracle  == 'U0':
        U = II
    if oracle == 'U1':
        U = bitflip_operator('1H')
    if oracle == 'U2':
        U = CNOT_operator()
    if oracle == 'U3':
        U = CNOT_operator() * bitflip_operator('1H')

    return hadamard_operator('both') * U * hadamard_operator('both')
    # # constant functions
    # if oracle == 'U0':
    #     return 
    # if oracle == 'U1':
    #     return hadamard_operator('both') * U(1) * hadamard_operator('both')
    # # balanced functions
    # if oracle == 'U2':
    #     return hadamard_operator('both') * U(2) * hadamard_operator('both')
    # if oracle == 'U3':
    #     return hadamard_operator('both') * U(3) * hadamard_operator('both')
 

In [18]:
P0 = II # do nothing 
P1 = Sx(np.pi/2) * IzSz(np.pi) * SyIx(np.pi/2) * IzSz(np.pi) * Iy(np.pi/2)
P2 = Ix(np.pi/2) * IzSz(np.pi) * SxIy(np.pi/2) * IzSz(np.pi) * Sy(np.pi/2)
def temporal_average(rho):
    P0 = II # do nothing 
    P1 = Sx(np.pi/2) * IzSz(np.pi) * SyIx(np.pi/2) * IzSz(np.pi) * Iy(np.pi/2)
    P2 = Ix(np.pi/2) * IzSz(np.pi) * SxIy(np.pi/2) * IzSz(np.pi) * Sy(np.pi/2)

    rho0 = P0 * rho * P0.dag()
    rho1 = P1 * rho * P1.dag()
    rho2 = P2 * rho * P2.dag()
    
    return (rho0 + rho1 + rho2)/3 + (5/3 * II)

def apply_gate_tmp_avg(rho, gate, state='00'):
    if state == '00':
        bitflip = II
    if state == '01':
        bitflip = bitflip_operator('13C')
    if state == '10':
        bitflip = bitflip_operator('1H')
    if state == '11':
        bitflip = bitflip_operator('both')
    

    rho_tmp_avg = temporal_average(rho)
    rho_state = bitflip * rho_tmp_avg * bitflip.dag()
    rho_gate_applied = gate * rho_state * gate.dag()

    return rho_gate_applied

#### Question 4: Grover's Algorithm

Following Appendix E in 'Quantum Information Processing with NMR' we use:
$$ G = H^{\otimes 2}PH^{\otimes 2}O. $$
Where we define oracle, O, as the CZ operator:
$$
O = \begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & -1
\end{bmatrix}
$$

and P adds a phase difference between solutions and non-solutions. For example in the |00> case:
$$
P = \begin{bmatrix}
1 & 0 & 0 & 0 \\
0 & -1 & 0 & 0 \\
0 & 0 & -1 & 0 \\
0 & 0 & 0 & -1
\end{bmatrix}
$$




In [19]:
def grover(U, solution):
    
    H = hadamard_operator('both')
    if solution == '11':
        P = bitflip_operator('both') * CZ_operator() * bitflip_operator('both')
    if solution == '10':
        P = bitflip_operator('1H') * CZ_operator()* bitflip_operator('1H')
    if solution == '01':
        P = bitflip_operator('13C') * CZ_operator()* bitflip_operator('13C')
    if solution == '00':
        P = CZ_operator()

    G =  H * U * H * P * H 

    return G

Applying Grover to the state |00> for different solution states:

In [20]:
oracle = CZ_operator() # O
print(apply_gate_tmp_avg(rho, grover(oracle, solution='00'), '00'))
print(apply_gate_tmp_avg(rho, grover(oracle, solution='01'), '00'))
print(apply_gate_tmp_avg(rho, grover(oracle, solution='10'), '00'))
print(apply_gate_tmp_avg(rho, grover(oracle, solution='11'), '00'))


Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[6.667 0.    0.    0.   ]
 [0.    0.    0.    0.   ]
 [0.    0.    0.    0.   ]
 [0.    0.    0.    0.   ]]
Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.    0.    0.    0.   ]
 [0.    6.667 0.    0.   ]
 [0.    0.    0.    0.   ]
 [0.    0.    0.    0.   ]]
Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.    0.    0.    0.   ]
 [0.    0.    0.    0.   ]
 [0.    0.    6.667 0.   ]
 [0.    0.    0.    0.   ]]
Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.    0.    0.    0.   ]
 [0.    0.    0.    0.   ]
 [0.    0.    0.    0.   ]
 [0.    0.    0.    6.667]]


We see the outputs here give us the correct solution

### Let's simulate all 3 algorithms

In [21]:
def simulate_algorithm(gate, initial_state='all'):
    rho = Qobj([[5,0,0,0],
            [0,3,0,0],
            [0,0,-3,0],
            [0,0,0,-5]], dims = [[2,2], [2,2]])
    
    if initial_state == 'all':
        states = ['00', '01', '10', '11']
    else: 
        states = [initial_state]
    
    for state in states:
        if state == '00':
            bitflip = II
        if state == '01':
            bitflip = bitflip_operator('13C')
        if state == '10':
            bitflip = bitflip_operator('1H')
        if state == '11':
            bitflip = bitflip_operator('both')

    # apply pulse sequence to get spectra peaks
        p0_spectra = evolve_density_matrix(rho, gate * bitflip * P0)
        # plot_spectra(p0_spectra, "P0")
        p1_spectra = evolve_density_matrix(rho, gate * bitflip * P1)
        # plot_spectra(p0_spectra, "P1")
        p2_spectra = evolve_density_matrix(rho, gate * bitflip * P2)
        # plot_spectra(p0_spectra, "P2")

    # sum peaks for temporal averaging
        sum_abcd = sum_P0_P1_P2(p0_spectra, p1_spectra, p2_spectra)

    # plot temporal average spectra (and get temporal averaged peak heights)
        peaks = plot_spectra(sum_abcd, f"Temporal Average |{state}>", show=False)
        
    # use peak heights from temporal average to recover a,b,c,d
        solve_linear_abcd(peaks, state)


In [22]:
# CNOT real
print("Simulating real CNOT")
simulate_algorithm(CNOT_operator())

Simulating real CNOT
Solving for a,b,c,d for|00> is approximately: [15. -5. -5. -5.]


Solving for a,b,c,d for|01> is approximately: [-5. 15. -5. -5.]
Solving for a,b,c,d for|10> is approximately: [-5. -5. -5. 15.]
Solving for a,b,c,d for|11> is approximately: [-5. -5. 15. -5.]


In [23]:
# Grover 
print("Simulating Grover")
oracle = CZ_operator() # O
simulate_algorithm(grover(oracle, solution='00'), '00')
simulate_algorithm(grover(oracle, solution='01'), '00')
simulate_algorithm(grover(oracle, solution='10'), '00')
simulate_algorithm(grover(oracle, solution='11'), '00')


Simulating Grover


Solving for a,b,c,d for|00> is approximately: [15. -5. -5. -5.]
Solving for a,b,c,d for|00> is approximately: [-5. 15. -5. -5.]
Solving for a,b,c,d for|00> is approximately: [-5. -5. 15. -5.]
Solving for a,b,c,d for|00> is approximately: [-5. -5. -5. 15.]


In [24]:
# DJ
print("Simulating DJ")
print("constant f(x) = 0")
simulate_algorithm(deutsch_josza('U0'), '00')
print("constant f(x) = 1")
simulate_algorithm(deutsch_josza('U1'), '00')
print("balanced f(x) = x")
simulate_algorithm(deutsch_josza('U2'), '00')
print("balanced f(x) = !x")
simulate_algorithm(deutsch_josza('U3'), '00')

Simulating DJ
constant f(x) = 0
Solving for a,b,c,d for|00> is approximately: [15. -5. -5. -5.]
constant f(x) = 1
Solving for a,b,c,d for|00> is approximately: [15. -5. -5. -5.]
balanced f(x) = x
Solving for a,b,c,d for|00> is approximately: [15. -5. -5. -5.]
balanced f(x) = !x


Solving for a,b,c,d for|00> is approximately: [15. -5. -5. -5.]
